# New section

In [1]:
import pandas as pd
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
!pip install flask_ngrok 
import flask
import difflib
from flask import Flask,request,render_template
from flask_ngrok import run_with_ngrok

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.


In [2]:
ratings=pd.read_csv('Ratings .csv')
movies=pd.read_csv('movies.csv')
userdata=pd.read_csv('user_data.csv')
d=pd.merge(userdata,ratings,on='UserID',how='left')
df=pd.merge(d,movies,on='MovieID',how='left')

# New section

***User Based Surprise SVD***

In [3]:
e = pd.read_csv('svd_p.csv')    ##Read predicted(USER BASED) dataset

In [4]:
#Recommend Top N to User
def recommend(userid_, n):
  f=e.loc[e[(e.uid==userid_)].est.sort_values(ascending=False).index][:n].iid.tolist()
  p=movies.loc[movies[movies.MovieID.isin(f)].index]
  k = ratings.loc[ratings[(ratings.UserID==userid_)].index]['MovieID'].tolist()
  return p.loc[p[~p.MovieID.isin(k)].index]

def rec(a,b):
  t = recommend(a,b)[['MovieID','Title','Genres']]
  t1,t2,t3 = t['MovieID'].to_list(),t['Title'].to_list(),t['Genres'].to_list()
  return t1,t2,t3


In [5]:
recommend(1,15)

,MovieID,Title,Genres
315,318,"Shawshank Redemption, The (1994)",Drama
589,593,"Silence of the Lambs, The (1991)",Drama|Thriller
891,903,Vertigo (1958),Mystery|Thriller
892,904,Rear Window (1954),Mystery|Thriller
908,920,Gone with the Wind (1939),Drama|Romance|War
941,953,It's a Wonderful Life (1946),Drama
1180,1198,Raiders of the Lost Ark (1981),Action|Adventure
1215,1234,"Sting, The (1973)",Comedy|Crime
1230,1250,"Bridge on the River Kwai, The (1957)",Drama|War
1252,1272,Patton (1970),Drama|War


***Content Based (Genres)***

In [6]:
#Split Movies Genres in str
movies['Genres_'] = movies['Genres'].str.split('|')
movies['Genres_'] = movies['Genres'].fillna("").astype('str')
movies.head(50)


tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0)
matrix = tf.fit_transform(movies['Genres_'])
csim = linear_kernel(matrix,matrix)

titles = movies['Title']
indices = pd.Series(movies.index, index=movies['Title'])

#Recommend Similar Movies
def c_rec(title):
    idx = indices[title]
    sim_sc = list(enumerate(csim[idx]))
    sim_sc = sorted(sim_sc, key=lambda x: x[1], reverse=True)[1:]
    movie_idx = [i[0] for i in sim_sc]
    print('Movies Similar to : ',title)
    yy=pd.DataFrame(movies.Title.iloc[movie_idx])[:10]
    return yy
c_rec('Dracula: Dead and Loving It (1995)')

Movies Similar to :  Dracula: Dead and Loving It (1995)


,Title
326,Tales from the Hood (1995)
726,Cemetery Man (Dellamorte Dellamore) (1994)
789,"Frighteners, The (1996)"
1221,Braindead (1992)
1235,Bad Taste (1987)
1258,Young Frankenstein (1974)
1310,April Fool's Day (1986)
1409,"Machine, The (1994)"
1934,Gremlins (1984)
1935,Gremlins 2: The New Batch (1990)


***CountVectorizer***

In [7]:
#fit and Trnasform the data into Vectorizer
cv = CountVectorizer()
cv_matrix = cv.fit_transform(movies['Genres'])
simc = cosine_similarity(cv_matrix, cv_matrix)

movies_ = movies.reset_index()
indices = pd.Series(movies_.index, index=movies_['Title'])
all_titles = [movies['Title'][i] for i in range(len(movies_['Title']))]

# Recommendation
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(simc[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    tit = movies['Title'].iloc[movie_indices]
    dat = movies['Genres'].iloc[movie_indices]
    return_df = pd.DataFrame(columns=['Title','Genres'])
    return_df['Title'] = tit
    return_df['Genres'] = dat
    return return_df
get_recommendations('Paths of Glory (1957)')

,Title,Genres
153,Beyond Rangoon (1995),Drama|War
332,"Walking Dead, The (1995)",Drama|War
523,Schindler's List (1993),Drama|War
641,Courage Under Fire (1996),Drama|War
967,Nothing Personal (1995),Drama|War
979,Michael Collins (1996),Drama|War
1074,Platoon (1986),Drama|War
1162,Paths of Glory (1957),Drama|War
1190,Apocalypse Now (1979),Drama|War
1199,Ran (1985),Drama|War


In [ ]:

app = flask.Flask(__name__, template_folder='templates')
run_with_ngrok(app)


@app.route('/', methods=['GET', 'POST'])
def index(): return render_template('front.html')


@app.route("/predict/",methods=["POST"])
def main():
  if request.method == 'POST': m_name = request.form['movie_name'].title()
  if request.method == "GET" : m_name = request.args.get('movie_name').title()
  

  print(m_name)
  if m_name not in all_titles: return(render_template('front.html'))
  else:
    result_final = get_recommendations(m_name)
    names = []
    dates = []
    for i in range(len(result_final)):
      names.append(result_final.iloc[i][0])
      dates.append(result_final.iloc[i][1])
    return flask.render_template('back.html',result = 1,ret = zip(names,dates),search_name=m_name)


@app.route("/user/",methods=["POST"])
def userPred():
  print(request.method)

  try: a = int(request.form['user_id'])
  except: return render_template('front.html')
  print(a)
  b = 30
  t1,t2,t3 = rec(a,b)
  return render_template('back.html',neut = 1,ret = zip(t1,t2,t3))

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
Exception in thread Thread-5:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/threading.py", line 1254, in run
    self.function(*self.args, **self.kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/flask_ngrok.py", line 70, in start_ngrok
    ngrok_address = _run_ngrok()
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/flask_ngrok.py", line 31, in _run_ngrok
    ngrok = subprocess.Popen([executable, 'http', '5000'])
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/subprocess.py", line 858, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/subp

Beyond Rangoon (1995)
